In [1]:
import os
import time
import random
import ipywidgets as widgets
from IPython.display import display, HTML
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIWWW
from collections import Counter
import pandas as pd
import primer3

# Set up Entrez email
Entrez.email = "ghorbani.abozar@gmail.com"

# Widget inputs
virus_name_input = widgets.Text(
    value='Tomato Mosaic Virus',
    description='Virus Name:',
    placeholder='Enter virus name',
    layout=widgets.Layout(width="400px")
)

output_dir_input = widgets.Text(
    value=r"C:\Users\AFRAA\OneDrive\Documents\result",
    description='Output Dir:',
    placeholder='Enter output directory',
    layout=widgets.Layout(width="400px")
)

download_btn = widgets.Button(description="Download Sequences", button_style="success")
align_btn = widgets.Button(description="Create Alignment & Contigs", button_style="info")
design_btn = widgets.Button(description="Design Primers", button_style="primary")
blast_btn = widgets.Button(description="Run Primer BLAST", button_style="danger")
output_box = widgets.Output()

# Function to download sequences
def download_all_sequences(virus_name, save_path):
    try:
        with output_box:
            output_box.clear_output()
            print(f"Downloading sequences for {virus_name}...")
        
        search_term = f"{virus_name}[Organism] AND genome[Title]"
        handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=10)
        record = Entrez.read(handle)
        handle.close()

        if record["Count"] == "0":
            print(f"No records found for {virus_name}.")
            return
        
        os.makedirs(save_path, exist_ok=True)

        for seq_id in record["IdList"]:
            handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()

            filename = os.path.join(save_path, f"{virus_name}_{seq_id}.fasta")
            SeqIO.write(seq_record, filename, "fasta")
            print(f"Sequence saved: {filename}")
    except Exception as e:
        print(f"Error downloading sequences: {e}")

# Function to create alignment and consensus
def create_alignment_and_contigs(input_folder, output_folder):
    try:
        with output_box:
            print("Creating consensus sequence...")
        
        input_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".fasta")]
        sequences = [SeqIO.read(f, "fasta") for f in input_files]

        max_length = max(len(seq) for seq in sequences)
        aligned_sequences = [str(seq.seq).ljust(max_length, '-') for seq in sequences]
        consensus_seq = ''.join(Counter(col).most_common(1)[0][0] for col in zip(*aligned_sequences))

        consensus_record = SeqRecord(Seq(consensus_seq), id="Consensus_Sequence", description="")
        output_file = os.path.join(output_folder, "Consensus_Sequence.fasta")
        SeqIO.write(consensus_record, output_file, "fasta")
        print(f"Consensus sequence saved to {output_file}")
    except Exception as e:
        print(f"Error creating consensus sequence: {e}")

# Function to design primers
def design_primers(fasta_path, output_path, num_primers=3, product_size_range=(250, 1000)):
    try:
        with output_box:
            print("Designing primers...")
        
        record = SeqIO.read(fasta_path, 'fasta')
        primer_pairs = []

        for i in range(num_primers):
            product_size = random.randint(*product_size_range)
            target_start = random.randint(0, len(record.seq) - product_size)
            target_end = target_start + product_size
            target_sequence = record.seq[target_start:target_end]

            primers = primer3.bindings.design_primers(
                {
                    'SEQUENCE_TEMPLATE': str(target_sequence),
                    'SEQUENCE_INCLUDED_REGION': [0, len(target_sequence)],
                },
                {
                    'PRIMER_OPT_SIZE': 20,
                    'PRIMER_MIN_SIZE': 18,
                    'PRIMER_MAX_SIZE': 25,
                    'PRIMER_MIN_TM': 58,
                    'PRIMER_MAX_TM': 65,
                    'PRIMER_PAIR_MAX_DIFF_TM': 2
                }
            )

            primer_pairs.append((primers['PRIMER_LEFT_0_SEQUENCE'], primers['PRIMER_RIGHT_0_SEQUENCE']))

        output_file = os.path.join(output_path, 'primer_pairs.txt')
        with open(output_file, 'w') as file:
            file.write("Forward Primer\tReverse Primer\n")
            for fp, rp in primer_pairs:
                file.write(f"{fp}\t{rp}\n")
        print(f"Primers saved to {output_file}")
    except Exception as e:
        print(f"Error designing primers: {e}")

# Function to run Primer-BLAST
def run_primer_blast(primer_pairs_file, output_dir):
    try:
        with open(primer_pairs_file, 'r') as file:
            primer_pairs = [line.strip().split('\t') for line in file.readlines()[1:]]

        os.makedirs(output_dir, exist_ok=True)

        for i, (fp, rp) in enumerate(primer_pairs):
            sequence = Seq(f'{fp}{rp}')
            seq_record = SeqRecord(sequence, id=f"Primer_Pair_{i}", description="")
            result_handle = NCBIWWW.qblast("blastn", "nr", seq_record.format("fasta"))
            with open(os.path.join(output_dir, f"primer_blast_result_{i}.xml"), "w") as out_file:
                out_file.write(result_handle.read())
            result_handle.close()
            print(f"BLAST results saved for Primer Pair {i}")
            time.sleep(2)
    except Exception as e:
        print(f"Error running Primer-BLAST: {e}")

# Event handlers
def on_download_click(b):
    download_all_sequences(virus_name_input.value, output_dir_input.value)

def on_align_click(b):
    create_alignment_and_contigs(output_dir_input.value, output_dir_input.value)

def on_design_click(b):
    design_primers(os.path.join(output_dir_input.value, "Consensus_Sequence.fasta"), output_dir_input.value)

def on_blast_click(b):
    run_primer_blast(os.path.join(output_dir_input.value, "primer_pairs.txt"), output_dir_input.value)

# Link buttons to handlers
download_btn.on_click(on_download_click)
align_btn.on_click(on_align_click)
design_btn.on_click(on_design_click)
blast_btn.on_click(on_blast_click)

# Layout
app = widgets.VBox([
    widgets.HTML("<h2>AutoPVPrimer Dashboard</h2>"),
    virus_name_input,
    output_dir_input,
    widgets.HBox([download_btn, align_btn, design_btn, blast_btn]),
    output_box
])

display(app)
